# Public Assistance Funded Projects Details - v1
Author: Mark Bauer

In [1]:
# import libraries
import duckdb
from datetime import datetime

In [2]:
# reproducibility
%reload_ext watermark
%watermark -v -p duckdb

Python implementation: CPython
Python version       : 3.11.0
IPython version      : 8.6.0

duckdb: 1.0.0



In [3]:
# data retrieved
current_date = datetime.now()
print(f"This notebook was executed on {current_date.strftime('%Y-%m-%d')}.")

This notebook was executed on 2024-10-06.


# OpenFEMA Dataset: Public Assistance Funded Projects Details - v1

## Dataset
Federal Emergency Management Agency (FEMA), OpenFEMA Dataset: Public Assistance Funded Projects Details - v1. Retrieved from https://www.fema.gov/openfema-data-page/public-assistance-funded-projects-details-v1. This product uses the FEMA OpenFEMA API, but is not endorsed by FEMA. The Federal Government or FEMA cannot vouch for the data or analyses derived from these data after the data have been retrieved from the Agency's website(s).

Read more about [OpenFEMA Terms and Conditions](https://www.fema.gov/about/openfema/terms-conditions).

## Dataset Description
>The Public Assistance Funded Projects Details dataset contains obligated (financial obligation to grantee) Public Assistance projects, lists public assistance recipients designated as applicants in the data, and a list of every funded, individual project, called project worksheets. Open projects still under pre-obligation processing are not represented. Applicant details can be obtained by relating this dataset with the OpenFEMA Public Assistance Applicants dataset using the Applicant ID field.

Source: [Public Assistance Funded Projects Details - v1](https://www.fema.gov/openfema-data-page/public-assistance-funded-projects-details-v1)

## Disaster Delcarations
For more information on the disaster declaration process:
- Information about disasters: https://www.fema.gov/disasters
- How a disaster is declared: https://www.fema.gov/disasters/how-declared

# Additional Resources
To examine other cool data visualizations about this data, visit FEMA's [Public Assistance Program Summary of Obligations](https://www.fema.gov/data-visualization/public-assistance-program-summary-obligations) page.

# Read In Data

In [4]:
# duckdb connection
con = duckdb.connect()

# create public assistance table
con.execute("""
    CREATE TABLE public_assistance AS
    FROM read_csv('https://www.fema.gov/api/open/v1/PublicAssistanceFundedProjectsDetails.csv')
"""
)

# sanity check
sql = """
    SELECT *
    FROM public_assistance
    LIMIT 10
"""

con.sql(sql)

┌────────────────┬─────────────────────┬──────────────┬───┬──────────────────────┬──────────────────────┐
│ disasterNumber │   declarationDate   │ incidentType │ … │         hash         │          id          │
│     int64      │      timestamp      │   varchar    │   │       varchar        │       varchar        │
├────────────────┼─────────────────────┼──────────────┼───┼──────────────────────┼──────────────────────┤
│           4023 │ 2011-09-02 00:00:00 │ Hurricane    │ … │ 5ff26ea21f9af4de5d…  │ d247f735-5e5d-4d47…  │
│           4155 │ 2013-11-08 00:00:00 │ Severe Storm │ … │ 39a1559d2dd1a04afa…  │ 39552c91-2dc1-4519…  │
│           4226 │ 2015-06-26 00:00:00 │ Severe Storm │ … │ aef455b4030e92d3c7…  │ d02b7bb6-fb6c-4fb4…  │
│           4314 │ 2017-05-22 00:00:00 │ Severe Storm │ … │ 727e5a2e3628d483b2…  │ 044e88f1-b38e-483c…  │
│           4337 │ 2017-09-10 00:00:00 │ Hurricane    │ … │ ab453e975b05cc1163…  │ 98731455-3b88-4fc0…  │
│           4480 │ 2020-03-20 00:00:00 │ Biolo

In [5]:
# pandas dataframe view
con.sql(sql).df().head()

,disasterNumber,declarationDate,incidentType,pwNumber,applicationTitle,applicantId,damageCategoryCode,projectSize,county,countyCode,...,stateNumberCode,projectAmount,federalShareObligated,totalObligated,obligatedDate,dcc,damageCategory,lastRefresh,hash,id
0,4023,2011-09-02,Hurricane,247,MCMDG00 Trumbull Ave Revetment,009-47500-00,G - Recreational or Other,Large,New Haven,9,...,9,614291.00,614291.00,614291.00,2012-04-05,G,Recreational or Other,2024-03-23 17:02:09.387,5ff26ea21f9af4de5dce40c1f4ab2324b11b5fcb,d247f735-5e5d-4d47-9b4b-58a2fa6396b4
1,4155,2013-11-08,Severe Storm,173,SG107 - Donated Resources,093-62100-00,B - Protective Measures,Small,Meade,93,...,46,20739.92,16506.90,16506.90,2024-03-22,B,Protective Measures,2024-03-23 17:02:09.387,39a1559d2dd1a04afafdcde601838082c65b4729,39552c91-2dc1-4519-87fb-b6beb815a0ad
2,4226,2015-06-26,Severe Storm,62,SCT002C - Roads and Culverts,127-99127-00,C - Roads and Bridges,Large,Scott,127,...,5,205754.79,154316.09,154316.09,2015-12-08,C,Roads and Bridges,2024-03-23 17:02:09.387,aef455b4030e92d3c7219138d3456fcebdd789bf,d02b7bb6-fb6c-4fb4-a029-58fdfc9848f9
3,4314,2017-05-22,Severe Storm,14,DUARW01 - PAAP Debris Removal 1-30 Days,051-20500-00,A - Debris Removal,Large,Holmes,51,...,28,2115399.61,1687334.81,1687334.81,2017-07-21,A,Debris Removal,2024-03-23 17:02:09.387,727e5a2e3628d483b2ac6407f48663a3a69a144e,044e88f1-b38e-483c-b579-80a60c6a10fa
4,4337,2017-09-10,Hurricane,6679,Zoo Exhibit and Trail Repairs,099-UXTQE-00,"G - Parks, Recreational Facilities, and Other ...",Large,Palm Beach County,99,...,12,137858.93,124077.54,124077.54,2020-01-24,G,"Parks, Recreational Facilities, and Other Items",2024-09-06 15:04:53.183,ab453e975b05cc11633455ccde7607fdd24ff1ca,98731455-3b88-4fc0-af7f-dae089378537


# Describe and Summarize Data

In [6]:
# count rows
con.sql("SELECT COUNT(id) AS count_rows FROM public_assistance")

┌────────────┐
│ count_rows │
│   int64    │
├────────────┤
│     796343 │
└────────────┘

In [7]:
# examine column datatypes
sql = "DESCRIBE SELECT * FROM public_assistance"
describe_relation = con.sql(sql)

(con
 .sql("SELECT column_name, column_type FROM describe_relation")
 .show(max_rows=50)
)

┌───────────────────────┬─────────────┐
│      column_name      │ column_type │
│        varchar        │   varchar   │
├───────────────────────┼─────────────┤
│ disasterNumber        │ BIGINT      │
│ declarationDate       │ TIMESTAMP   │
│ incidentType          │ VARCHAR     │
│ pwNumber              │ BIGINT      │
│ applicationTitle      │ VARCHAR     │
│ applicantId           │ VARCHAR     │
│ damageCategoryCode    │ VARCHAR     │
│ projectSize           │ VARCHAR     │
│ county                │ VARCHAR     │
│ countyCode            │ BIGINT      │
│ state                 │ VARCHAR     │
│ stateCode             │ VARCHAR     │
│ stateNumberCode       │ BIGINT      │
│ projectAmount         │ DOUBLE      │
│ federalShareObligated │ DOUBLE      │
│ totalObligated        │ DOUBLE      │
│ obligatedDate         │ TIMESTAMP   │
│ dcc                   │ VARCHAR     │
│ damageCategory        │ VARCHAR     │
│ lastRefresh           │ TIMESTAMP   │
│ hash                  │ VARCHAR     │


In [8]:
# summary statistics
sql = "SUMMARIZE SELECT * FROM public_assistance"
summarize_relation = con.sql(sql)

summarize_relation.df()

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,disasterNumber,BIGINT,1239,4828,1651,2635.9586635407104,1278.8717355436274,1604,1861,4136,796343,0.00
1,declarationDate,TIMESTAMP,1998-08-26 00:00:00,2024-09-28 00:00:00,1266,None,None,None,None,None,796343,0.00
2,incidentType,VARCHAR,Biological,Winter Storm,28,None,None,None,None,None,796343,0.00
3,pwNumber,BIGINT,1,99995,20071,1655.6118092329561,2814.074128091725,191,596,1719,796343,0.00
4,applicationTitle,VARCHAR,\t1-30 day debris removal,"¿Public Health Division 14, July 6, 2021 thru ...",527159,None,None,None,None,None,796343,0.78
5,applicantId,VARCHAR,000-00000-01,999-99999-99,67665,None,None,None,None,None,796343,0.00
6,damageCategoryCode,VARCHAR,A - Debris Removal,Z - State Management,16,None,None,None,None,None,796343,0.00
7,projectSize,VARCHAR,Large,Small,2,None,None,None,None,None,796343,1.63
8,county,VARCHAR,Abbeville,Ziebach,3556,None,None,None,None,None,796343,1.70
9,countyCode,BIGINT,0,66010,334,72.46648751811166,200.0134074455639,14,54,105,796343,1.63


In [9]:
# examine NULL percentage
sql = """
    SELECT column_name, column_type, null_percentage
    FROM summarize_relation
    ORDER BY null_percentage DESC
"""

con.sql(sql).show(max_rows=50)

┌───────────────────────┬─────────────┬─────────────────┐
│      column_name      │ column_type │ null_percentage │
│        varchar        │   varchar   │  decimal(9,2)   │
├───────────────────────┼─────────────┼─────────────────┤
│ county                │ VARCHAR     │            1.70 │
│ projectSize           │ VARCHAR     │            1.63 │
│ countyCode            │ BIGINT      │            1.63 │
│ applicationTitle      │ VARCHAR     │            0.78 │
│ disasterNumber        │ BIGINT      │            0.00 │
│ declarationDate       │ TIMESTAMP   │            0.00 │
│ incidentType          │ VARCHAR     │            0.00 │
│ pwNumber              │ BIGINT      │            0.00 │
│ applicantId           │ VARCHAR     │            0.00 │
│ damageCategoryCode    │ VARCHAR     │            0.00 │
│ state                 │ VARCHAR     │            0.00 │
│ stateCode             │ VARCHAR     │            0.00 │
│ stateNumberCode       │ BIGINT      │            0.00 │
│ projectAmoun

In [10]:
# examine declarationDate
sql = """
    SELECT *
    FROM summarize_relation
    WHERE column_name = 'declarationDate'
"""

con.sql(sql).df()

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,declarationDate,TIMESTAMP,1998-08-26 00:00:00,2024-09-28 00:00:00,1266,None,None,None,None,None,796343,0.0


In [11]:
# examine lastRefresh
sql = """
    SELECT *
    FROM summarize_relation
    WHERE column_name = 'lastRefresh'
"""

con.sql(sql).df()

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,lastRefresh,TIMESTAMP,2024-03-22 11:47:01.426,2024-10-06 15:05:23.825,134,None,None,None,None,None,796343,0.0


In [12]:
# examine federalObligatedAmount
sql = """
    SELECT *
    FROM summarize_relation
    WHERE column_name IN ('projectAmount', 'federalShareObligated', 'totalObligated')
"""

con.sql(sql).df()

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,projectAmount,DOUBLE,-372687108.0,9053782360.01,599176,337427.0639684902,15386481.714057699,3948.7887692359386,12003.220031898703,43009.79611193659,796343,0.0
1,federalShareObligated,DOUBLE,-372687108.0,9053782360.01,624636,306791.9878754184,14510651.054568479,3135.6542150898094,9626.449390926728,34846.218617027815,796343,0.0
2,totalObligated,DOUBLE,-376423296.25,9053782360.01,659122,307562.26090598304,14515016.285539614,3232.5651512343948,9881.17765497742,35762.26143051343,796343,0.0


# Preview Values

In [13]:
# preview data
sql = """
    SELECT *
    FROM public_assistance
    LIMIT 5   
"""

con.sql(sql).df().iloc[:, 10:]

,state,stateCode,stateNumberCode,projectAmount,federalShareObligated,totalObligated,obligatedDate,dcc,damageCategory,lastRefresh,hash,id
0,Connecticut,CT,9,614291.00,614291.00,614291.00,2012-04-05,G,Recreational or Other,2024-03-23 17:02:09.387,5ff26ea21f9af4de5dce40c1f4ab2324b11b5fcb,d247f735-5e5d-4d47-9b4b-58a2fa6396b4
1,South Dakota,SD,46,20739.92,16506.90,16506.90,2024-03-22,B,Protective Measures,2024-03-23 17:02:09.387,39a1559d2dd1a04afafdcde601838082c65b4729,39552c91-2dc1-4519-87fb-b6beb815a0ad
2,Arkansas,AR,5,205754.79,154316.09,154316.09,2015-12-08,C,Roads and Bridges,2024-03-23 17:02:09.387,aef455b4030e92d3c7219138d3456fcebdd789bf,d02b7bb6-fb6c-4fb4-a029-58fdfc9848f9
3,Mississippi,MS,28,2115399.61,1687334.81,1687334.81,2017-07-21,A,Debris Removal,2024-03-23 17:02:09.387,727e5a2e3628d483b2ac6407f48663a3a69a144e,044e88f1-b38e-483c-b579-80a60c6a10fa
4,Florida,FL,12,137858.93,124077.54,124077.54,2020-01-24,G,"Parks, Recreational Facilities, and Other Items",2024-09-06 15:04:53.183,ab453e975b05cc11633455ccde7607fdd24ff1ca,98731455-3b88-4fc0-af7f-dae089378537


In [14]:
# count per declaration and applicant
sql = """
    SELECT
       disasterNumber,
       state,
       applicantId,
       applicationTitle,
       pwNumber,
       county,
       count(id) AS count
    FROM
        public_assistance
    GROUP BY ALL
    ORDER BY count DESC, disasterNumber ASC  
"""

relation = con.sql(sql)

con.sql("""
    SELECT *
    FROM relation
    ORDER BY count DESC
    LIMIT 20
""").df()

,disasterNumber,state,applicantId,applicationTitle,pwNumber,county,count
0,1294,Pennsylvania,045-76792-00,PUMPING STATION,304,Delaware,2
1,1334,North Dakota,079-80020-00,EMERGENCY WORK FOR EAGLES VIEW & TURTLEVILLE H...,1295,Rolette,2
2,1446,Guam,000-UU5GZ-00,EMERGENCY WORK,1220,Statewide,2
3,1539,Florida,015-0332A-00,PUBLIC BUILDINGS AND FACILITIES,4268,Charlotte,2
4,1551,Florida,033-0517F-00,Net Small Project Cost Overrun (NSPO),3855,Escambia,2
5,1571,Alaska,050-78245-00,SHED REPAIR,99,Bethel (CA),2
6,1577,California,059-1C6FF-00,RESERVOIR REPAIR- TALEGA RESERVOIR,1538,Orange,2
7,1577,California,000-U8RA6-00,PARK REPAIR- PT. MUGU STATE PARK,1069,Statewide,2
8,1577,California,037-30000-00,EMERGENCY PROTECTIVE MEASURES - CAMINO SAN RAF...,2859,Los Angeles,2
9,1577,California,111-UL4GF-00,DAM REPAIR- ARUNDELL BARRANCA,1989,Ventura,2


The data suggests that one Disaster Number and Applicant can have multiple records in this dataset. Let's examine further.

In [15]:
# examine count unique for applicantTitle = 'PUMPING STATION' for disaster 1294
(con
 .sql("""
    SELECT *
    FROM public_assistance
    WHERE applicationTitle = 'PUMPING STATION'
        AND disasterNumber = 1294
""")
 .df()
 .describe(include='object')
 .loc['unique']
)

incidentType          1
applicationTitle      1
applicantId           1
damageCategoryCode    2
projectSize           1
county                1
state                 1
stateCode             1
dcc                   2
damageCategory        2
hash                  2
id                    2
Name: unique, dtype: object

In [16]:
# count per declaration and applicant
sql = """
    SELECT
       disasterNumber,
       state,
       applicantId,
       applicationTitle,
       pwNumber,
       projectSize,
       damageCategory,
       county,
       count(id) AS count
    FROM
        public_assistance
    GROUP BY ALL
    ORDER BY count DESC, disasterNumber ASC  
"""

relation = con.sql(sql)

con.sql("""
    SELECT *
    FROM relation
    ORDER BY count DESC
    LIMIT 20
""").df()

,disasterNumber,state,applicantId,applicationTitle,pwNumber,projectSize,damageCategory,county,count
0,3175,Massachusetts,021-03367-00,"SNOW REMOVAL / MA, CT, NH",5,Small,Protective Measures,Norfolk,1
1,3175,Massachusetts,003-71095-00,SNOW REMOVAL,24,Small,Protective Measures,Berkshire,1
2,3175,Massachusetts,013-00765-00,SNOW REMOVAL,29,Large,Protective Measures,Hampden,1
3,3175,Massachusetts,017-27480-00,SNOW REMOVAL,31,Small,Protective Measures,Middlesex,1
4,3175,Massachusetts,013-28075-00,SNOW REMOVAL,32,Small,Protective Measures,Hampden,1
5,3175,Massachusetts,015-27690-00,SNOW REMOVAL,56,Small,Protective Measures,Hampshire,1
6,3175,Massachusetts,021-25065-00,SNOW REMOVAL,65,Large,Protective Measures,Norfolk,1
7,3175,Massachusetts,011-35180-00,SNOW REMOVAL,126,Small,Protective Measures,Franklin,1
8,3175,Massachusetts,023-57565-00,SNOW REMOVAL,140,Small,Protective Measures,Plymouth,1
9,3175,Massachusetts,007-21150-00,SNOW REMOVAL,174,Small,Protective Measures,Dukes,1


From the analysis above, unique record consist of a disaster, a state, an applicant, PW number, project size, damage category, and a county.

# Analysis
`WHERE declarationDate >= '2003-01-01'`

In [17]:
con.sql("""
    SELECT *
    FROM public_assistance
    LIMIT 5
""").df()

,disasterNumber,declarationDate,incidentType,pwNumber,applicationTitle,applicantId,damageCategoryCode,projectSize,county,countyCode,...,stateNumberCode,projectAmount,federalShareObligated,totalObligated,obligatedDate,dcc,damageCategory,lastRefresh,hash,id
0,4023,2011-09-02,Hurricane,247,MCMDG00 Trumbull Ave Revetment,009-47500-00,G - Recreational or Other,Large,New Haven,9,...,9,614291.00,614291.00,614291.00,2012-04-05,G,Recreational or Other,2024-03-23 17:02:09.387,5ff26ea21f9af4de5dce40c1f4ab2324b11b5fcb,d247f735-5e5d-4d47-9b4b-58a2fa6396b4
1,4155,2013-11-08,Severe Storm,173,SG107 - Donated Resources,093-62100-00,B - Protective Measures,Small,Meade,93,...,46,20739.92,16506.90,16506.90,2024-03-22,B,Protective Measures,2024-03-23 17:02:09.387,39a1559d2dd1a04afafdcde601838082c65b4729,39552c91-2dc1-4519-87fb-b6beb815a0ad
2,4226,2015-06-26,Severe Storm,62,SCT002C - Roads and Culverts,127-99127-00,C - Roads and Bridges,Large,Scott,127,...,5,205754.79,154316.09,154316.09,2015-12-08,C,Roads and Bridges,2024-03-23 17:02:09.387,aef455b4030e92d3c7219138d3456fcebdd789bf,d02b7bb6-fb6c-4fb4-a029-58fdfc9848f9
3,4314,2017-05-22,Severe Storm,14,DUARW01 - PAAP Debris Removal 1-30 Days,051-20500-00,A - Debris Removal,Large,Holmes,51,...,28,2115399.61,1687334.81,1687334.81,2017-07-21,A,Debris Removal,2024-03-23 17:02:09.387,727e5a2e3628d483b2ac6407f48663a3a69a144e,044e88f1-b38e-483c-b579-80a60c6a10fa
4,4337,2017-09-10,Hurricane,6679,Zoo Exhibit and Trail Repairs,099-UXTQE-00,"G - Parks, Recreational Facilities, and Other ...",Large,Palm Beach County,99,...,12,137858.93,124077.54,124077.54,2020-01-24,G,"Parks, Recreational Facilities, and Other Items",2024-09-06 15:04:53.183,ab453e975b05cc11633455ccde7607fdd24ff1ca,98731455-3b88-4fc0-af7f-dae089378537


## Statistics

In [18]:
sql = """
    SELECT
        ROUND(SUM(totalObligated), 0)::BIGINT AS totalObligated
    FROM public_assistance
    WHERE declarationDate >= '2003-01-01'
"""

con.sql(sql)

┌────────────────┐
│ totalObligated │
│     int64      │
├────────────────┤
│   236057593222 │
└────────────────┘

In [19]:
# format dollar value
totalObligated = con.sql(sql).df().values[0][0]

print(f"Total public assistance program federal obligated amount: ${totalObligated:,}")

Total public assistance program federal obligated amount: $236,057,593,222


In [20]:
sql = """
    SELECT
        COUNT(DISTINCT disasterNumber)
    FROM public_assistance
    WHERE declarationDate >= '2003-01-01'
"""

disasterNumber = con.sql(sql).df().values[0][0]

print(f"Total public assistance program declarations: {disasterNumber:,}")

Total public assistance program declarations: 1,455


In [21]:
sql = """
    SELECT
        COUNT(DISTINCT applicantId)
    FROM public_assistance  
    WHERE declarationDate >= '2003-01-01'
"""

applicantName = con.sql(sql).df().values[0][0]

print(f"Number of applicants with funded projects: {applicantName:,}")

Number of applicants with funded projects: 60,642


## Tables

**Table xx.** Top 20 Disaster Declarations, States, Years, and Incident Types with the Highest Total Obligated Amount

In [22]:
sql = """
    SELECT
        disasterNumber,
        state,
        substring(declarationDate::VARCHAR, 1, 4) as year,
        incidentType,
        ROUND(SUM(totalObligated), 0)::BIGINT AS totalObligated
    FROM
        public_assistance
    WHERE declarationDate >= '2003-01-01'    
    GROUP BY ALL
    ORDER BY totalObligated DESC   
"""

relation = con.sql(sql)

con.sql("""
    SELECT *
    FROM relation
    ORDER BY totalObligated DESC
    LIMIT 20
""").df()

,disasterNumber,state,year,incidentType,totalObligated
0,4339,Puerto Rico,2017,Hurricane,33898115751
1,4480,New York,2020,Biological,15592369461
2,4340,Virgin Islands,2017,Hurricane,15586263768
3,4485,Texas,2020,Biological,14931375168
4,4085,New York,2012,Hurricane,14740187736
5,1603,Louisiana,2005,Hurricane,13668079220
6,4482,California,2020,Biological,11830450424
7,4486,Florida,2020,Biological,3693646921
8,1604,Mississippi,2005,Hurricane,3218622940
9,4496,Massachusetts,2020,Biological,2999138790


**Table xx.** Number and Total Amount Obligations by Damage Category

In [23]:
sql = """
    SELECT
        damageCategoryCode,
        COUNT(totalObligated) AS countObligated,
        ROUND(SUM(totalObligated), 0)::BIGINT AS totalObligated
    FROM
        public_assistance
    WHERE declarationDate >= '2003-01-01'
    GROUP BY damageCategoryCode  
"""

relation = con.sql(sql)

con.sql("""
    SELECT *
    FROM relation
    ORDER BY damageCategoryCode ASC
""").df()

,damageCategoryCode,countObligated,totalObligated
0,A - Debris Removal,74154,19140132347
1,B - Emergency Protective Measures,55287,99449331437
2,B - Emergency Work Donated Resources,1682,45500530
3,B - Protective Measures,144431,13253760722
4,C - Roads and Bridges,215918,15657081483
5,D - Water Control Facilities,9632,2340808010
6,E - Buildings and Equipment,18339,17996270605
7,E - Public Buildings,74183,18169310717
8,F - Public Utilities,31091,11124660454
9,F - Utilities,7371,22218493148


**Table xx.** Top 20 Years with the Highest Total Obligated Amount

In [24]:
sql = """
    SELECT
        substring(declarationDate::VARCHAR, 1, 4) as year,
        ROUND(SUM(totalObligated), 0)::BIGINT AS totalObligated
    FROM
        public_assistance
    WHERE declarationDate >= '2003-01-01'    
    GROUP BY year
    ORDER BY totalObligated DESC   
"""

relation = con.sql(sql)

con.sql("""
    SELECT *
    FROM relation
    ORDER BY totalObligated DESC
    LIMIT 20
""").df()

,year,totalObligated
0,2020,92438329738
1,2017,57454564223
2,2005,21511199782
3,2012,17889151347
4,2018,6344152801
5,2008,5946622930
6,2021,5152550494
7,2022,4542739814
8,2011,3819025442
9,2004,3465609239


**Table xx.** Incident Types with the Highest Total Obligated Amount

In [25]:
sql = """
    SELECT
        incidentType,
        ROUND(SUM(totalObligated), 0)::BIGINT AS totalObligated
    FROM
        public_assistance
    WHERE declarationDate >= '2003-01-01'    
    GROUP BY incidentType
    ORDER BY totalObligated DESC   
"""

relation = con.sql(sql)

con.sql("""
    SELECT *
    FROM relation
    ORDER BY totalObligated DESC
""").df()

,incidentType,totalObligated
0,Hurricane,115926268623
1,Biological,85443415142
2,Severe Storm,12211142820
3,Flood,5142268150
4,Wildfire,3770552402
5,Severe Storm(s),3710141167
6,Tropical Storm,1838047348
7,Fire,1543299553
8,Earthquake,1423440480
9,Severe Ice Storm,1238587018


**Table xx.** Top 20 Years and Incident Types with the Highest Total Obligated Amount

In [26]:
sql = """
    SELECT
        substring(declarationDate::VARCHAR, 1, 4) as year,
        incidentType,
        ROUND(SUM(totalObligated), 0)::BIGINT AS totalObligated
    FROM
        public_assistance
    WHERE declarationDate >= '2003-01-01'    
    GROUP BY 1, 2
    ORDER BY totalObligated DESC   
"""

relation = con.sql(sql)

con.sql("""
    SELECT *
    FROM relation
    ORDER BY totalObligated DESC
    LIMIT 20
""").df()

,year,incidentType,totalObligated
0,2020,Biological,85440521701
1,2017,Hurricane,54990567125
2,2005,Hurricane,20669013377
3,2012,Hurricane,17490241387
4,2022,Hurricane,3441944891
5,2020,Hurricane,3429696550
6,2008,Hurricane,3294989443
7,2018,Hurricane,3282368557
8,2004,Hurricane,2932842864
9,2021,Hurricane,2842386127


**Table xx.** Top 20 States with the Highest Total Obligated Amount

In [27]:
sql = """
    SELECT
        state,
        ROUND(SUM(totalObligated), 0)::BIGINT AS totalObligated
    FROM
        public_assistance
    WHERE declarationDate >= '2003-01-01'    
    GROUP BY state
    ORDER BY totalObligated DESC   
"""

relation = con.sql(sql)

con.sql("""
    SELECT *
    FROM relation
    ORDER BY totalObligated DESC
    LIMIT 20
""").df()

,state,totalObligated
0,Puerto Rico,36462536660
1,New York,33586303403
2,Louisiana,24757619937
3,Texas,22166397427
4,California,18247802530
5,Florida,16580845499
6,Virgin Islands,15812459002
7,New Jersey,5963211725
8,Mississippi,4229728856
9,Massachusetts,3707518089


**Table xx.** Top 20 States and Incident Types with the Highest Total Obligated Amount

In [28]:
sql = """
    SELECT
        state,
        incidentType,
        ROUND(SUM(totalObligated), 0)::BIGINT AS totalObligated
    FROM
        public_assistance
    WHERE declarationDate >= '2003-01-01'    
    GROUP BY 1, 2
    ORDER BY totalObligated DESC   
"""

relation = con.sql(sql)

con.sql("""
    SELECT *
    FROM relation
    ORDER BY totalObligated DESC
    LIMIT 20
""").df()

,state,incidentType,totalObligated
0,Puerto Rico,Hurricane,35212034071
1,Louisiana,Hurricane,21187039504
2,Virgin Islands,Hurricane,15653648522
3,New York,Biological,15595931648
4,New York,Hurricane,15306829137
5,Texas,Biological,14931375168
6,Florida,Hurricane,12408060638
7,California,Biological,11839818729
8,Texas,Hurricane,6190444436
9,Florida,Biological,3695456566
